# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bardai,11.6500,7.2333,32.55,15,34,5.95,NG,1712774632
1,1,ushuaia,-54.8000,-68.3000,8.79,71,40,13.38,AR,1712774582
2,2,puerto natales,-51.7236,-72.4875,10.25,53,0,7.20,CL,1712774615
3,3,namibe,-15.1961,12.1522,22.21,72,0,3.54,AO,1712774787
4,4,duminichi,53.9345,35.1099,16.00,69,86,2.06,RU,1712774787


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
import holoviews as hv
from holoviews.element.tiles import OSM

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",             # Adjust column name for longitude if necessary
    "Lat",             # Adjust column name for latitude if necessary
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",   # Size of the point represents humidity
    color="City",      # Use 'City' column for colors
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7          # Set transparency to 0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_humidity



c:\Users\Sarah Son Kim\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Filter cities based on criteria (e.g., temperature between 20°C and 25°C, humidity below 50%, cloudiness below 30%, wind speed below 5 m/s)
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 50) & 
                                  (city_data_df['Wind Speed'] < 5)].copy()

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,sabha,27.0377,14.4283,23.93,16,17,4.76,LY,1712774791
81,81,de witt,41.8234,-90.5382,20.74,41,0,2.57,US,1712774795
455,455,awjilah,29.1081,21.2869,20.95,25,0,3.76,LY,1712774685
480,480,mucumpiz,8.4167,-71.1333,23.53,45,46,2.97,VE,1712774838


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
40,sabha,LY,27.0377,14.4283,16,
81,de witt,US,41.8234,-90.5382,41,
455,awjilah,LY,29.1081,21.2869,25,
480,mucumpiz,VE,8.4167,-71.1333,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000  # Define the search radius in meters
params = {
    "categories": "accommodation.hotel",
    #"radius": radius,
    "limit": 40,  
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f":proximity: {lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df.head()



Starting hotel search
sabha - nearest hotel: No hotel found
de witt - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
mucumpiz - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
40,sabha,LY,27.0377,14.4283,16,No hotel found
81,de witt,US,41.8234,-90.5382,41,No hotel found
455,awjilah,LY,29.1081,21.2869,25,No hotel found
480,mucumpiz,VE,8.4167,-71.1333,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot with hotel names and country information in the hover message
map_with_hotel_info = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],  # Add hotel name and country to hover message
    color="City",
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_with_hotel_info


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)